# Audit Extension

This notebook has the calculation of audit caracteristics from the database. It combines the excelsheet with the SIM data.
It also calculates percentiles global and internal industry, and store it extending the q_industries, and then store it in the memory.

At the bottom, this notebook constructs the q_industries data, which are selected industries with quarterly frequency.



In [3]:
import pandas as pd
import numpy as np
from dstools.mlutils.corp_tax_audit_unsupervised import (
                    apply_time_index_from_Y_and_Q, 
                    load_industry_data, 
                    clean_data
)

from dstools.mlutils.corp_tax_sea_audits_treatment import (apply_format_filtering_and_change_status,
                                                           get_number_of_quarters_between_dates,
                                                           create_nquarter_list_between_dates
                                                          )

ModuleNotFoundError: No module named 'dstools.mlutils.corp_tax_sea_audits_treatment'

In [ ]:
# %store

# Preparing the merging of audits

In [ ]:
# Restoring percentile classified SLIM data
%store -r q_industries
q_industries.columns

In [ ]:
naics_codes = list(q_industries['naics2'].unique())
naics_codes

In [ ]:
#delete
audit_data.head()

In [ ]:
# PREPARE THE DATA freq = Q
# Monthly has been excluded because is much less data, and less audits
# code from similarity_score_research.ipynb
path_audit_data ='~/paola_notebooks/Unsupervised_Analysis/Resources/SEA-audit-labeled-cit-cleaned-with-periods.csv'
audit_threshold = 500

# Audit Data - reformatting to be able to merge with unsupervised dataset
#naics = naics_code_used
# Loading the clean audit data
audit_data = pd.read_csv(path_audit_data, parse_dates = True, infer_datetime_format=True)

# Adding succesful status, plus format changes to sync with sqldata, plus filtering by industry
audit_data = apply_format_filtering_and_change_status(audit_data_from_csv = audit_data, naics_code_used = None,  audit_threshold = audit_threshold)

## Adding period lenght by number of quarters
audit_data['nquarters'] = get_number_of_quarters_between_dates(audit_data, first_period_col = 'first_period', last_period_col='last_period')

## Generating new index for the audit data
pro_index_df = create_nquarter_list_between_dates(df=audit_data, start_column = 'first_period', end_column='last_period' )

## Meging new index to the audit data
audit_data_ext = pro_index_df.merge(audit_data, left_on=['original_index'], right_index=True, how='outer', copy=True)
audit_data_ext.index = pd.PeriodIndex(audit_data_ext.date, freq='Q')
audit_data_ext.drop(columns=['date'], inplace=True)

audit_data_ext.head()

In [6]:
# Q - OBTAIN THE AUDIT DATA WITH COMBINED FEATURES, BUT ONLY AUDIT DATA
print("Quarterly companies")
audit_data_combined = audit_data_ext.reset_index().merge(q_industries, how='left', on=['date','business_id'])
# print(audit_data_combined.shape)
# print(audit_data_combined.isnull().sum())
#Filter by naics codes used - naics2 will be strings for the industries
audit_data_combined = audit_data_combined[audit_data_combined['naics2'].isin(naics_codes)]
audit_data_combined_clean = audit_data_combined.dropna()
print(f"shape pre-drop:{audit_data_combined['business_id'].unique().shape}")
print(f"shape after-drop:{audit_data_combined_clean['business_id'].unique().shape}")

audit_business_id = audit_data_combined['business_id'].unique()
audit_business_id_change = audit_data_combined[abs(audit_data_combined['assessment_amount'])>500]['business_id'].unique()
audit_business_id_pos_change = audit_data_combined[audit_data_combined['assessment_amount']>500]['business_id'].unique()

print(f" Q audited busineses: {len(audit_business_id)}")
print(f" Q change audit busineses: {len(audit_business_id_change)}")
print(f" Q positive change audit busineses: {len(audit_business_id_pos_change)}")

audit_business_id_clean = audit_data_combined_clean['business_id'].unique()
audit_business_id_change_clean = audit_data_combined_clean[abs(audit_data_combined['assessment_amount'])>500]['business_id'].unique()
audit_business_id_pos_change_clean = audit_data_combined_clean[audit_data_combined['assessment_amount']>500]['business_id'].unique()

print(f" Clean audited busineses: {len(audit_business_id_clean)}")
print(f" Clean change audit busineses: {len(audit_business_id_change_clean)}")
print(f" Clean positive change audit busineses: {len(audit_business_id_pos_change_clean)}")

Quarterly companies


NameError: name 'audit_data_ext' is not defined

In [ ]:
q_audit_data_combined = audit_data_combined_clean
q_audit_data_combined['eff_tax%_perc_glob'].describe(), q_audit_data_combined['deduc2income_perc_glob'].describe()

In [ ]:
q_audit_data_combined[['change+','change-','k-cluster_g', 'k-cluster_ind']]
q_audit_data_combined[['naics2','change+','change-','k-cluster_g', 'k-cluster_ind']].groupby('naics2').sum()

In [ ]:
type(q_industries['is_ccluster_ind'][0])

In [ ]:
# LABEL OF POSITIVE CHANGES OUTSIDE THE COMPLIANT CLUSTER
q_audit_data_combined['ischange+_outside_ccluster_ind'] = ((q_audit_data_combined['change+']) &
                                                      (q_audit_data_combined['is_ccluster_ind']==0)
                                                      ).astype(int)
q_audit_data_combined['ischange+OR-_outside_ccluster_ind'] = ((q_audit_data_combined['change+'] | q_audit_data_combined['change-']) &
                                                         (q_audit_data_combined['is_ccluster_ind']==0)
                                                        ).astype(int)

print("Cases with at least one of the positive changes, global or ind, it is outside the compliant cluster")
q_audit_data_combined[(q_audit_data_combined['ischange+_outside_ccluster']+q_audit_data_combined['ischange+OR-_outside_ccluster'])>0]

In [ ]:
q_audit_data_combined['ischange+_outside_ccluster'].value_counts()

In [ ]:
q_audit_data_combined['ischange+OR-_outside_ccluster'].value_counts()

In [ ]:
# Active from Raw to Code if wanting to update current store
%store q_audit_data_combined

In [5]:
q_audit_data_combined.columns
#IT DOESN'T NEED DATE INDEX

Index(['date', 'original_index', 'business_id', 'assessment_amount', 'naics_2',
       'entity_name', 'first_period', 'last_period', 'change', 'change+',
       'change-', 'nquarters', 'index', 'naics2', 'bus_loc_id', 'naics_code',
       'business_legal_name', 'maxNumLoc', 'tax_period_cd', 'obl_type_id',
       'sumsum_gross', 'sumsum_deduc', 'sumsum_taxable', 'sumsum_paid',
       'eff_tax_rate%', 'deduc_to_income_ratio', 'eff_tax%_perc_glob',
       'eff_tax%_perc_ind', 'deduc2income_perc_glob', 'deduc2income_perc_ind',
       '4Delta_abs_eff_tax_rate%', '4Delta_abs_deduc_to_income_ratio',
       '4D_eff_tax%_perc_glob', '4D_deduc2income_perc_glob',
       '4D_eff_tax_perc_ind', '4D_deduc2income_perc_ind', 'k-cluster_ind',
       'k-cluster_g', 'sumsum_gross_perc_glob', 'sumsum_gross_perc_ind',
       'ischange+_outside_ccluster', 'ischange+OR-_outside_ccluster',
       'is_ccluster_ind', 'is_ccluster_g', 'ischange+_outside_ccluster_ind',
       'ischange+OR-_outside_ccluster_ind'],

In [10]:
%store -r q_industries
q_industries.columns

Index(['index', 'naics2', 'business_id', 'bus_loc_id', 'naics_code',
       'business_legal_name', 'maxNumLoc', 'tax_period_cd', 'obl_type_id',
       'sumsum_gross', 'sumsum_deduc', 'sumsum_taxable', 'sumsum_paid',
       'eff_tax_rate%', 'deduc_to_income_ratio', 'eff_tax%_perc_glob',
       'eff_tax%_perc_ind', 'deduc2income_perc_glob', 'deduc2income_perc_ind',
       '4Delta_abs_eff_tax_rate%', '4Delta_abs_deduc_to_income_ratio',
       '4D_eff_tax%_perc_glob', '4D_deduc2income_perc_glob',
       '4D_eff_tax_perc_ind', '4D_deduc2income_perc_ind', 'k-cluster_ind',
       'k-cluster_g', 'sumsum_gross_perc_glob', 'sumsum_gross_perc_ind',
       'is_ccluster_ind', 'is_ccluster_g', 'max_outlier_score', 'hd-cluster',
       'out_of_compliance', 'prop_outside_compliant_cluster'],
      dtype='object')

# Adding new features to existing q_audit_data_combined

In [20]:
# Adding new features to existing audit_data_combined
q_audit_data_combined = q_audit_data_combined.reset_index().merge(q_industries.reset_index(drop=True)[['date','business_id','out_of_compliance', 'prop_outside_compliant_cluster']], how='left', on=['date','business_id'])

In [23]:
q_audit_data_combined.drop(columns = ['level_0','original_index', 'index' ], inplace=True)
q_audit_data_combined.describe()

,assessment_amount,naics_2,nquarters,bus_loc_id,maxNumLoc,obl_type_id,sumsum_gross,sumsum_deduc,sumsum_taxable,sumsum_paid,...,sumsum_gross_perc_glob,sumsum_gross_perc_ind,ischange+_outside_ccluster,ischange+OR-_outside_ccluster,is_ccluster_ind,is_ccluster_g,ischange+_outside_ccluster_ind,ischange+OR-_outside_ccluster_ind,out_of_compliance,prop_outside_compliant_cluster
count,7121.000000,7121.000000,7121.000000,7121.000000,7121.000000,7121.000000,7.121000e+03,7.121000e+03,7.121000e+03,7121.000000,...,7121.000000,7121.000000,7121.000000,7121.000000,7121.000000,7121.000000,7121.000000,7121.000000,7121.000000,7121.000000
mean,2298.473467,44.103497,17.144502,412627.426204,3.382952,11.474091,1.324280e+07,7.170300e+06,6.072499e+06,18104.206126,...,0.833783,0.819696,0.173992,0.248139,0.591490,0.462575,0.173992,0.248139,0.408510,0.344079
std,61287.288645,15.610609,4.094877,254147.627052,29.138652,13.211894,4.275994e+07,3.839353e+07,1.430325e+07,38716.081157,...,0.220070,0.224203,0.379130,0.431963,0.491593,0.498632,0.379130,0.431963,0.491593,0.364497
min,-558940.010000,23.000000,1.000000,118.000000,1.000000,10.000000,1.469730e+03,0.000000e+00,0.000000e+00,0.000000,...,0.001267,0.000368,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,33.000000,16.000000,139671.000000,1.000000,10.000000,9.501320e+05,0.000000e+00,3.317057e+05,1021.430000,...,0.790723,0.767431,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,208.430000,42.000000,17.000000,546533.000000,1.000000,10.000000,3.177898e+06,2.477300e+04,1.537806e+06,4848.410000,...,0.924779,0.908085,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.218391
75%,5807.820000,54.000000,17.000000,610159.000000,1.000000,10.000000,8.996152e+06,1.884101e+06,5.583790e+06,15803.960000,...,0.974301,0.970754,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.656250
max,374374.310000,81.000000,56.000000,757155.000000,582.000000,133.000000,5.421126e+08,5.395872e+08,1.946442e+08,426731.600000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
%store q_audit_data_combined

Stored 'q_audit_data_combined' (DataFrame)
